<a href="https://colab.research.google.com/github/salma-abed/Deep-learning-based-automated-detection-and-classification-of-Alzheimer-s-disease-Using-Neuroimaging/blob/main/Medical_image_slicer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from PIL import Image
import nibabel as nib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/ADNI_nii_after_pre'

In [ ]:
class ImageSlicer:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.labels = ['AD', 'CN', 'LMCI', 'MCI']
        self.output_folder = '/content/drive/MyDrive/output_images_preprocessed'
        self.slices_range = range(10)

    def slice_images(self):
        for label in self.labels:
            input_folder = os.path.join(self.dataset_path, label)
            output_folder = os.path.join(self.dataset_path, self.output_folder, label)
            os.makedirs(output_folder, exist_ok=True)

            for filename in os.listdir(input_folder):
                if filename.endswith('.nii'):
                    filepath = os.path.join(input_folder, filename)
                    nii_image = nib.load(filepath)
                    np_image = np.array(nii_image.get_fdata())[..., np.newaxis]  # add an additional dimension along axis=-1
                    np_image = np.squeeze(np_image, axis=-1)  # squeeze the array along axis=-1
                    np_image = np_image.astype(np.float32)
                    min_val = np.min(np_image)
                    max_val = np.max(np_image)
                    np_image = ((np_image - min_val) * (255 / (max_val - min_val))).astype(np.uint8)
                    num_slices = np_image.shape[2]
                    middle_slice = num_slices // 2  # get the middle slice index
                    start_slice = max(0, middle_slice - 5)  # get the starting slice index
                    end_slice = min(num_slices - 1, middle_slice + 4)  # get the ending slice index
                    folder_name = os.path.splitext(filename)[0] + '_folder_0'
                    folder_path = os.path.join(output_folder, folder_name)
                    os.makedirs(folder_path, exist_ok=True)
                    for i in range(start_slice, end_slice+1):
                        slice_image = Image.fromarray(np_image[:, :, i])
                        slice_image = slice_image.convert('RGB')  # convert to RGB mode
                        slice_filename = f'slice_{i}.jpg'
                        slice_filepath = os.path.join(folder_path, slice_filename)
                        slice_image.save(slice_filepath)


In [ ]:
# create an instance of the ImageSlicer class with the path to the dataset
slicer = ImageSlicer(dataset_path)

# call the slice_images method to slice and store the images
slicer.slice_images()
